# GreenFlow Rabat — Smart Traffic Intelligence
### Hackathon RamadnIA 2026

**Objective:** Transform existing CCTV cameras into smart traffic sensors using Edge AI.

**Pipeline Overview:**
1. Extract frames from local Rabat driving footage
2. Annotate with Roboflow Label Assist (4 classes)
3. Train YOLO11n with small-object optimizations
4. Export to ONNX/NCNN for Raspberry Pi 5 / Jetson
5. Real-time RTSP inference → JSON webhook → n8n orchestration

**Classes:** `License Plate` · `Car` · `Grand Taxi` · `Triporteur`

**Privacy:** 100% local processing. No images leave the edge device.

## Phase 1: Environment Setup
Installs all dependencies and verifies the hardware (CPU vs GPU).  
Run this cell once per session.

In [5]:
# ---------- Install Dependencies ----------
# We use subprocess instead of !pip to work on ANY platform
# (local, Colab, Kaggle, Paperspace — no Colab-specific syntax).

import subprocess, sys

packages = [
    "ultralytics",            # YOLO11n — our detection model
    "roboflow",               # Dataset download + annotation API
    "opencv-python-headless", # Image/video processing (headless = no GUI needed)
    "requests",               # HTTP calls to n8n webhooks
]

for pkg in packages:
    subprocess.check_call(
        [sys.executable, "-m", "pip", "install", "-q", pkg],
        stdout=subprocess.DEVNULL  # suppress noisy pip output
    )

print("All packages installed.")

All packages installed.


In [6]:
# ---------- Hardware Check ----------
# This tells us if we have a GPU available.
# Training on CPU = hours.  Training on GPU = minutes.

import torch
import platform

print("=" * 45)
print("  GreenFlow Rabat — Environment Report")
print("=" * 45)
print(f"  Python :  {platform.python_version()}")
print(f"  PyTorch:  {torch.__version__}")
print(f"  CUDA   :  {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"  GPU    :  {torch.cuda.get_device_name(0)}")
    print(f"  VRAM   :  {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("  GPU    :  None (CPU mode)")
    print("  Tip    :  Connect Colab GPU for training (Phase 5)")

print(f"  OS     :  {platform.system()} {platform.machine()}")
print("=" * 45)

  GreenFlow Rabat — Environment Report
  Python :  3.12.12
  PyTorch:  2.9.0+cpu
  CUDA   :  False
  GPU    :  None (CPU mode)
  Tip    :  Connect Colab GPU for training (Phase 5)
  OS     :  Linux x86_64


## Phase 2: Frame Extraction
Extract 1 frame every 5 seconds from the Rabat driving video.  
Output: ~480 JPEG images saved to `data/frames/` for Roboflow annotation.

In [7]:
# ---------- Configuration ----------
# We mount Google Drive to access the video AND save frames there.
# Everything stays on Drive — permanent, survives session restarts.
#
# WORKFLOW: For each video, change VIDEO_PATH and PREFIX, then re-run cells 6+7.
#   Run 1: day    → "rabat_day_0000.jpg"   (already done with 523 frames)
#   Run 2: night  → "rabat_night_0000.jpg"
#   Run 3: rain   → "rabat_rain_0000.jpg"
#   Run 4: dusk   → "rabat_dusk_0000.jpg"

from pathlib import Path
from google.colab import drive

# Mount Google Drive (a browser popup will ask for permission once)
drive.mount("/content/drive")

# ---- CHANGE THESE TWO FOR EACH VIDEO ----
VIDEO_PATH = "/content/drive/MyDrive/GreenFlow-Rabat/YTDown.com_YouTube_Rabat-4K-Night-Drive-Driving-Downtown-Re_Media_aFtBUlb9yj8_001_1080p.mp4"
PREFIX = "rabat_night"  # Change per video: "rabat_day", "rabat_night", "rabat_rain", "rabat_dusk"
# ------------------------------------------

# Extracted frames saved to your existing Drive folder (permanent storage)
OUTPUT_DIR = Path("/content/drive/MyDrive/GreenFlow-Rabat/Extracted_Frames")
INTERVAL_SEC = 5  # Extract 1 frame every N seconds

# Sanity check
import os
if os.path.exists(VIDEO_PATH):
    size_mb = os.path.getsize(VIDEO_PATH) / 1e6
    print(f"Video found: ({size_mb:.0f} MB)")
    print(f"Prefix: {PREFIX}_XXXX.jpg")
else:
    print(f"Video NOT found at: {VIDEO_PATH}")
    print("Check the filename in your Drive folder and update VIDEO_PATH above.")

Mounted at /content/drive
Video found: (1457 MB)
Prefix: rabat_night_XXXX.jpg


In [4]:
# ---------- Extract Frames ----------
import cv2

# Create output folder if it doesn't exist
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Open the video file
cap = cv2.VideoCapture(VIDEO_PATH)

# Read video metadata
fps = cap.get(cv2.CAP_PROP_FPS)                    # Frames per second of the video
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # Total number of frames
duration_min = total_frames / fps / 60              # Video length in minutes
frame_step = int(fps * INTERVAL_SEC)                # How many frames to skip between captures

print(f"Video info: {fps:.1f} FPS | {total_frames:,} total frames | {duration_min:.1f} min")
print(f"Strategy:   1 frame every {INTERVAL_SEC}s = every {frame_step} frames")
print(f"Expected:   ~{total_frames // frame_step} frames to extract\n")

# Main extraction loop
saved = 0
frame_idx = 0

while True:
    ret, frame = cap.read()       # Read one frame from the video
    if not ret:                   # ret is False when video ends
        break
    
    if frame_idx % frame_step == 0:  # Only save every Nth frame
        # Mask watermark "STREET.MA" at bottom-right corner
        h, w = frame.shape[:2]
        frame[h-120:h, w-450:w] = 0  # Black out a 450x120 px rectangle
        
        filename = OUTPUT_DIR / f"{PREFIX}_{saved:04d}.jpg"
        cv2.imwrite(str(filename), frame)
        saved += 1
        
        # Progress update every 50 saved frames
        if saved % 50 == 0:
            print(f"  Saved {saved} frames...")
    
    frame_idx += 1

cap.release()  # Always release the video capture when done

print(f"\nDone! Extracted {saved} frames → {OUTPUT_DIR}/")

Video info: 30.0 FPS | 95,040 total frames | 52.9 min
Strategy:   1 frame every 5s = every 149 frames
Expected:   ~637 frames to extract

  Saved 50 frames...
  Saved 100 frames...
  Saved 150 frames...
  Saved 200 frames...
  Saved 250 frames...
  Saved 300 frames...
  Saved 350 frames...
  Saved 400 frames...
  Saved 450 frames...
  Saved 500 frames...
  Saved 550 frames...
  Saved 600 frames...

Done! Extracted 638 frames → /content/drive/MyDrive/GreenFlow-Rabat/Extracted_Frames/


## Phase 2.5: Automated Frame Cleaning
Smart filtering to remove near-duplicates, severely blurred, and pitch-black frames.  
**Safety rule:** Nothing is deleted — rejected frames are *moved* to a `rejected_frames/` folder for manual review.

| Filter | Method | Day | Night | Rain |
|--------|--------|-----|-------|------|
| Near-duplicates | Perceptual hash (hamming dist) | Main filter | Active | Active |
| Motion blur | Laplacian variance | Strict (50) | Lenient (15) | Moderate (25) |
| Extreme darkness | Mean pixel intensity | Rarely triggers | Main filter | Rarely triggers |

**Targets:** 522→~480 Day · 637→~320 Night · 300→~230 Rain ≈ **~1,000 clean frames**

In [13]:
# ---------- Install imagehash (one-time) ----------
# imagehash provides perceptual hashing — it converts images into
# compact "fingerprints" so we can detect near-duplicates by comparing
# fingerprints instead of pixel-by-pixel (which is too slow and fragile).

import subprocess, sys

subprocess.check_call(
    [sys.executable, "-m", "pip", "install", "-q", "imagehash"],
    stdout=subprocess.DEVNULL
)
print("imagehash installed ✓")

imagehash installed ✓


In [ ]:
# ---------- Configuration & Folder Setup ----------
# Source: your existing Extracted_Frames folder on Drive
# Output: two NEW sibling folders — kept_frames/ and rejected_frames/
# Each mirrors the Day/Night/Rain subfolder structure.

import shutil
from pathlib import Path
from google.colab import drive

drive.mount("/content/drive")

# ---- Paths (adjust if your folder names differ) ----
BASE_DIR = Path("/content/drive/MyDrive/GreenFlow-Rabat")
SOURCE_DIR = BASE_DIR / "Extracted_Frames"     # Where all 1,459 frames live now
KEPT_DIR   = BASE_DIR / "kept_frames"           # Clean frames go here → Roboflow
REJECT_DIR = BASE_DIR / "rejected_frames"       # Rejected frames go here → manual review

# Subfolder mapping: prefix pattern → condition label
# This tells the script which threshold profile to use for each image.
#
# IMPORTANT: Day frames are named "rabat_0000.jpg" (no 'day' keyword),
# while night/rain have explicit prefixes ("rabat_night_", "rabat_rain_").
# We use "exclude" to prevent the day matcher from grabbing night/rain files.
CONDITIONS = {
    "day":   {"prefix": "rabat_",      "exclude": ["rabat_night", "rabat_rain"], "blur_thresh": 50, "dark_thresh": 20},
    "night": {"prefix": "rabat_night",  "exclude": [],                             "blur_thresh": 15, "dark_thresh": 30},
    "rain":  {"prefix": "rabat_rain",   "exclude": [],                             "blur_thresh": 25, "dark_thresh": 20},
}

# Similarity threshold: perceptual hash hamming distance.
# Two images with hash distance <= this value are considered duplicates.
# Lower = stricter (only very similar rejected). Higher = more aggressive.
HASH_DISTANCE_THRESH = 4   # ~94% similarity on a 64-bit hash

# Create output folders
for condition in CONDITIONS:
    (KEPT_DIR / condition).mkdir(parents=True, exist_ok=True)
    (REJECT_DIR / condition).mkdir(parents=True, exist_ok=True)

print(f"Source:   {SOURCE_DIR}")
print(f"Kept:     {KEPT_DIR}")
print(f"Rejected: {REJECT_DIR}")
print(f"\nSubfolders created: {list(CONDITIONS.keys())}")
print(f"Hash distance threshold: {HASH_DISTANCE_THRESH}")
for c, cfg in CONDITIONS.items():
    print(f"  {c:6s} → blur < {cfg['blur_thresh']}  |  dark < {cfg['dark_thresh']}")

SyntaxError: invalid syntax (ipython-input-693915021.py, line 47)

In [ ]:
# ---------- Filtering Functions ----------
# Each function returns True if the image FAILS the quality check
# (i.e., should be rejected).

import cv2
import numpy as np
import imagehash
from PIL import Image

def compute_blur_score(img_bgr):
    """
    Variance of the Laplacian — the classic blur detector.
    
    HOW IT WORKS:
    - The Laplacian operator detects edges (rapid intensity changes).
    - A sharp image has many strong edges → high variance.
    - A blurry image has few/weak edges → low variance.
    
    Returns a float: higher = sharper. Typical ranges:
      - Very sharp:  200+
      - Acceptable:  50–200
      - Blurry:      < 50  (day)  /  < 15 (night)
    """
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    laplacian = cv2.Laplacian(gray, cv2.CV_64F)  # 2nd derivative of intensity
    return laplacian.var()                         # Variance = spread of edge strengths


def is_too_blurry(img_bgr, threshold):
    """Check if frame is too blurry for the given condition's threshold."""
    score = compute_blur_score(img_bgr)
    return score < threshold, score


def is_too_dark(img_bgr, threshold):
    """
    Check if frame is too dark to contain useful information.
    
    HOW IT WORKS:
    - Convert to grayscale (single channel, 0=black, 255=white).
    - Compute the mean pixel value across the entire image.
    - If the average is below the threshold, the frame is essentially
      pitch-black or so dark that no vehicle features are distinguishable.
    
    Night frames naturally have lower means (~40–80), so the threshold
    is set per-condition to avoid false rejections.
    """
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    mean_intensity = np.mean(gray)
    return mean_intensity < threshold, mean_intensity


def compute_phash(img_path):
    """
    Compute a perceptual hash (pHash) for an image.
    
    HOW IT WORKS:
    - Resize image to 32x32, convert to grayscale.
    - Apply DCT (Discrete Cosine Transform) — like a 2D frequency analysis.
    - Keep only the low-frequency components (overall structure, not detail).
    - Threshold to produce a 64-bit binary hash.
    
    Two visually similar images produce nearly identical hashes,
    even if they differ in brightness, compression, or minor shifts.
    
    Hamming distance = number of bits that differ between two hashes.
    - 0 = identical images
    - 1-4 = near-duplicates (traffic jam consecutive frames)
    - 10+ = clearly different scenes
    """
    pil_img = Image.open(str(img_path))
    return imagehash.phash(pil_img)


def are_duplicates(hash1, hash2, threshold):
    """
    Compare two perceptual hashes.
    The '-' operator gives the Hamming distance (number of differing bits).
    """
    return (hash1 - hash2) <= threshold

print("Filtering functions loaded ✓")
print(f"  compute_blur_score(img)   → float (higher = sharper)")
print(f"  is_too_blurry(img, thr)   → (bool, score)")
print(f"  is_too_dark(img, thr)     → (bool, mean_intensity)")
print(f"  compute_phash(path)       → imagehash object")
print(f"  are_duplicates(h1,h2,thr) → bool")

Filtering functions loaded ✓
  compute_blur_score(img)   → float (higher = sharper)
  is_too_blurry(img, thr)   → (bool, score)
  is_too_dark(img, thr)     → (bool, mean_intensity)
  compute_phash(path)       → imagehash object
  are_duplicates(h1,h2,thr) → bool


In [ ]:
# ---------- Main Cleaning Loop ----------
# For each condition (day/night/rain):
#   1. Copy ALL frames from Extracted_Frames → kept_frames/{condition}/
#   2. Check darkness → move failures to rejected_frames/{condition}/
#   3. Check blur → move failures to rejected_frames/{condition}/
#   4. Check near-duplicates (consecutive) → move duplicates to rejected_frames/{condition}/
#
# ORDER MATTERS: We check darkness first (cheapest), then blur,
# then duplicates (most expensive — requires hashing every remaining image).

import time

# Stats tracking
stats = {c: {"total": 0, "dark": 0, "blur": 0, "dup": 0, "kept": 0} for c in CONDITIONS}

start_time = time.time()

for condition, cfg in CONDITIONS.items():
    print(f"\n{'='*55}")
    print(f"  Processing: {condition.upper()}")
    print(f"  Blur threshold: {cfg['blur_thresh']}  |  Dark threshold: {cfg['dark_thresh']}")
    print(f"{'='*55}")
    
    prefix = cfg["prefix"]
    blur_thresh = cfg["blur_thresh"]
    dark_thresh = cfg["dark_thresh"]
    
    kept_dir = KEPT_DIR / condition
    reject_dir = REJECT_DIR / condition
    
    # Step 1: Gather all matching frames from source
    # Sort by name so consecutive frames are adjacent (important for duplicate detection)
    #
    # Day files are "rabat_0000.jpg" — startswith("rabat_") would also match
    # "rabat_night_" and "rabat_rain_", so we explicitly exclude those.
    exclude = cfg.get("exclude", [])
    frames = sorted([
        f for f in SOURCE_DIR.iterdir()
        if f.name.startswith(prefix)
        and f.suffix.lower() in (".jpg", ".jpeg", ".png")
        and not any(f.name.startswith(ex) for ex in exclude)
    ])
    
    stats[condition]["total"] = len(frames)
    print(f"  Found {len(frames)} frames matching '{condition}' pattern")
    
    if len(frames) == 0:
        print(f"  ⚠ No frames found! Check that PREFIX matches your filenames.")
        continue
    
    # Step 2: Copy all frames to kept_frames first (non-destructive)
    print(f"  Copying to kept_frames/{condition}/ ...")
    for f in frames:
        shutil.copy2(str(f), str(kept_dir / f.name))
    
    # Now work exclusively inside kept_dir — move rejects out of it
    kept_files = sorted(list(kept_dir.iterdir()))
    
    # ----- Pass 1: Darkness Filter -----
    print(f"  Pass 1/3: Checking darkness (threshold: mean < {dark_thresh}) ...")
    dark_removed = 0
    surviving = []
    
    for fp in kept_files:
        img = cv2.imread(str(fp))
        if img is None:
            # Corrupted/unreadable file — reject it
            shutil.move(str(fp), str(reject_dir / fp.name))
            dark_removed += 1
            continue
        
        too_dark, mean_val = is_too_dark(img, dark_thresh)
        if too_dark:
            shutil.move(str(fp), str(reject_dir / fp.name))
            dark_removed += 1
        else:
            surviving.append(fp)
    
    stats[condition]["dark"] = dark_removed
    print(f"    Rejected {dark_removed} dark frames. Remaining: {len(surviving)}")
    
    # ----- Pass 2: Blur Filter -----
    print(f"  Pass 2/3: Checking blur (threshold: variance < {blur_thresh}) ...")
    blur_removed = 0
    still_alive = []
    
    for fp in surviving:
        img = cv2.imread(str(fp))
        too_blurry, blur_score = is_too_blurry(img, blur_thresh)
        if too_blurry:
            shutil.move(str(fp), str(reject_dir / fp.name))
            blur_removed += 1
        else:
            still_alive.append(fp)
    
    stats[condition]["blur"] = blur_removed
    print(f"    Rejected {blur_removed} blurry frames. Remaining: {len(still_alive)}")
    
    # ----- Pass 3: Near-Duplicate Filter -----
    print(f"  Pass 3/3: Checking near-duplicates (hash distance ≤ {HASH_DISTANCE_THRESH}) ...")
    dup_removed = 0
    
    if len(still_alive) > 1:
        # Compute hash for the first image
        prev_hash = compute_phash(still_alive[0])
        
        for i in range(1, len(still_alive)):
            curr_hash = compute_phash(still_alive[i])
            
            if are_duplicates(prev_hash, curr_hash, HASH_DISTANCE_THRESH):
                # Current frame is too similar to previous — reject it
                shutil.move(str(still_alive[i]), str(reject_dir / still_alive[i].name))
                dup_removed += 1
                # DON'T update prev_hash — keep comparing against the last KEPT frame
            else:
                # Different enough — keep it, update reference hash
                prev_hash = curr_hash
            
            # Progress update
            if i % 100 == 0:
                print(f"    Checked {i}/{len(still_alive)} frames...")
    
    stats[condition]["dup"] = dup_removed
    
    # Count final kept
    final_kept = len(list(kept_dir.iterdir()))
    stats[condition]["kept"] = final_kept
    
    print(f"    Rejected {dup_removed} near-duplicates.")
    print(f"  ✓ {condition.upper()} done: {final_kept} kept / {stats[condition]['total']} original")

elapsed = time.time() - start_time
print(f"\n{'='*55}")
print(f"  All conditions processed in {elapsed:.1f}s")
print(f"{'='*55}")


  Processing: DAY
  Blur threshold: 50  |  Dark threshold: 20
  Found 0 frames with prefix 'rabat_day'
  ⚠ No frames found! Check that PREFIX matches your filenames.

  Processing: NIGHT
  Blur threshold: 15  |  Dark threshold: 30
  Found 638 frames with prefix 'rabat_night'
  Copying to kept_frames/night/ ...


KeyboardInterrupt: 

In [ ]:
# ---------- Cleaning Summary Report ----------
# Final check: count actual files in each folder to confirm results.

print("\n" + "=" * 65)
print("  GREENFLOW RABAT — FRAME CLEANING REPORT")
print("=" * 65)
print(f"  {'Condition':<10} {'Start':>7} {'Dark':>6} {'Blur':>6} {'Dupes':>7} {'Kept':>6}  {'Target':>10}")
print(f"  {'-'*10:<10} {'-'*7:>7} {'-'*6:>6} {'-'*6:>6} {'-'*7:>7} {'-'*6:>6}  {'-'*10:>10}")

targets = {"day": "450–500", "night": "300–350", "rain": "200–250"}
total_start = 0
total_kept = 0
total_rejected = 0

for condition in CONDITIONS:
    s = stats[condition]
    rejected = s["dark"] + s["blur"] + s["dup"]
    total_start += s["total"]
    total_kept += s["kept"]
    total_rejected += rejected
    
    # Verify by actually counting files on disk
    actual_kept = len(list((KEPT_DIR / condition).iterdir()))
    actual_rejected = len(list((REJECT_DIR / condition).iterdir()))
    
    status = "✓" if actual_kept == s["kept"] else "⚠"
    
    print(f"  {condition.upper():<10} {s['total']:>7} {s['dark']:>6} {s['blur']:>6} {s['dup']:>7} {actual_kept:>6}  {targets[condition]:>10} {status}")

print(f"  {'-'*10:<10} {'-'*7:>7} {'-'*6:>6} {'-'*6:>6} {'-'*7:>7} {'-'*6:>6}  {'-'*10:>10}")
print(f"  {'TOTAL':<10} {total_start:>7} {'':>6} {'':>6} {'':>7} {total_kept:>6}  {'~1,000':>10}")
print("=" * 65)

# Actionable next steps
print(f"\n  📂 Kept frames:     {KEPT_DIR}")
print(f"  📂 Rejected frames: {REJECT_DIR}")
print(f"\n  NEXT STEPS:")
print(f"  1. Browse rejected_frames/ — rescue any good frames back to kept_frames/")
print(f"  2. Browse kept_frames/ — remove any remaining junk the script missed")
print(f"  3. When satisfied → Phase 3: Upload kept_frames/ to Roboflow")